# Data Cleaning Project – Mixed Orders Dataset

In this project, I practice core data cleaning techniques using a fictional mixed orders dataset. The dataset contains common data quality issues like:

- `price`: Missing values and wrong use of other currencies
- `order_date`: Missing values and no pattern
- `status`: Wrong formatting
- `city_state`: Separate city and state

This type of task is essential in real-world data projects, where raw data must be cleaned before analysis or modeling.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('pedidos_mistos.csv')

In [3]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   order_id    7 non-null      int64 
 1   price       6 non-null      object
 2   order_date  6 non-null      object
 3   status      7 non-null      object
 4   city_state  7 non-null      object
dtypes: int64(1), object(4)
memory usage: 412.0+ bytes


,order_id,price,order_date,status,city_state
0,12345,"R$ 199,99",01-04-2024,Em Trânsito,São Paulo - SP
1,12346,USD 180.00,2024/04/02,Entregue,"Rio de Janeiro,RJ"
2,12347,R$150.50,3 Abr 2024,Cancelado,Curitiba - PR
3,12348,"€210,75",04.04.2024,Pendente,Salvador-BA
4,12349,NaN,2024-04-05,entregue,"Recife , PE"
5,12350,R$ NaN,NaN,Cancelado,Belo Horizonte - MG
6,12351,199,06/04/24,EM TRÂNSITO,Fortaleza - CE


Problems:
- `price`: Missing values and wrong use of other currencies
- `order_date`: Missing values and no pattern
- `status`: Wrong formatting
- `city_state`: Separate city and state

### `price`

In [4]:
# First ill remove the letters from the `price` column and convert it to numeric
df['price'] = df['price'].str.replace(r'USD|R\$|\$|€| ','', regex=True) # Removed currency symbols, because they are all in USD

In [5]:
# Now I will convert the `price` column to float, replacing commas with dots
df['price'] = df['price'].str.replace(',','.').astype(float).round(2)
df['price']

0    199.99
1    180.00
2    150.50
3    210.75
4       NaN
5       NaN
6    199.00
Name: price, dtype: float64

### `order_date`

In [6]:
# order_date is all in a different format, need to fix the pattern
df['order_date_clean'] = df['order_date'].str.replace(r'-|\.| ', '/', regex=True)
df['order_date_clean']

0    01/04/2024
1    2024/04/02
2    3/Abr/2024
3    04/04/2024
4    2024/04/05
5           NaN
6      06/04/24
Name: order_date_clean, dtype: object

In [7]:
# Ill make a dictionary to map the Portuguese month abbreviations to English
# so I can standardize the date format
months = {
    'Jan': 'Jan', 'Fev': 'Feb', 'Mar': 'Mar', 'Abr': 'Apr',
    'Mai': 'May', 'Jun': 'Jun', 'Jul': 'Jul', 'Ago': 'Aug',
    'Set': 'Sep', 'Out': 'Oct', 'Nov': 'Nov', 'Dez': 'Dec'}

# Here I will replace the Portuguese month abbreviations with English ones
for pt, en in months.items():
    df['order_date_clean'] = df['order_date_clean'].str.replace(pt, en)

# And i will use a function to parse the dates in different formats, that i already used in the previous notebook
def parse_date(date_str):
    try:
        # It tries to parse as year-month-day
        return pd.to_datetime(date_str, format='%Y-%m-%d')
    except ValueError:
        # If it fails, it tries day/month/year
        return pd.to_datetime(date_str, dayfirst=True)

df['order_date'] = df['order_date_clean'].apply(parse_date)
df = df.drop('order_date_clean', axis=1)


In [8]:
df

,order_id,price,order_date,status,city_state
0,12345,199.99,2024-04-01,Em Trânsito,São Paulo - SP
1,12346,180.00,2024-02-04,Entregue,"Rio de Janeiro,RJ"
2,12347,150.50,2024-04-03,Cancelado,Curitiba - PR
3,12348,210.75,2024-04-04,Pendente,Salvador-BA
4,12349,NaN,2024-05-04,entregue,"Recife , PE"
5,12350,NaN,NaT,Cancelado,Belo Horizonte - MG
6,12351,199.00,2024-04-06,EM TRÂNSITO,Fortaleza - CE


### `status`

In [9]:
# The formating of the `status` column is inconsistent, so I will standardize it
df['status'] = df['status'].str.title().str.strip()
df['status']

0    Em Trânsito
1       Entregue
2      Cancelado
3       Pendente
4       Entregue
5      Cancelado
6    Em Trânsito
Name: status, dtype: object

### `city_state`

In [10]:
# Here ill separate the `city_state` column into `city` and `state`
df['city_state'] = df['city_state'].str.replace(r'-', ',', regex=True)
df[['city', 'state']] = df['city_state'].str.split(',', expand=True)
df[['city', 'state']]

,city,state
0,São Paulo,SP
1,Rio de Janeiro,RJ
2,Curitiba,PR
3,Salvador,BA
4,Recife,PE
5,Belo Horizonte,MG
6,Fortaleza,CE


In [11]:
# And to make sure there is no blank spaces in the `city` and `state` columns
df['city'] = df['city'].str.strip()
df['state'] = df['state'].str.strip()

# Just to make sure it worked, I will check the length of the `state` column
df['state'].str.len()

0    2
1    2
2    2
3    2
4    2
5    2
6    2
Name: state, dtype: int64

# Summary of Cleaning

- Cleaned `price` (fixed its Type and correct currency)  
- Cleaned `email` (standardized to US format and standardized it)  
- Cleaned `status` (fixed its format)
- Cleaned `city_state` (separated into 2 columns, `city` and `state`)

The cleaned dataset can now be saved and used in future analysis.

In [ ]:
df.to_csv('mixed_order_clean.csv', index=False)